# RNN with LSTM cell on IMDB







### Imports

In [ ]:
import torch
import torch.nn.functional as F
import torchtext 
import time
import random

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

### Settings and Dataset

In [ ]:
# Device
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


# Hyperparameters
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)
VOCABULARY_SIZE = 20000
LEARNING_RATE = 1e-4
BATCH_SIZE = 128
NUM_EPOCHS = 10
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
OUTPUT_DIM = 1

In [ ]:
TEXT = torchtext.legacy.data.Field(tokenize="basic_english",
                  include_lengths=True) 
LABEL = torchtext.legacy.data.LabelField(dtype=torch.float)


train_data, test_data = torchtext.legacy.datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state=random.seed(RANDOM_SEED),
                                          split_ratio=0.8)


print(f'Num Train: {len(train_data)}')
print(f'Num Valid: {len(valid_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 20000
Num Valid: 5000
Num Test: 25000


In [ ]:
TEXT.build_vocab(train_data, max_size=VOCABULARY_SIZE)
LABEL.build_vocab(train_data)


print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 20002
Number of classes: 2


In [ ]:
train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    sort_within_batch=True,
    device=DEVICE)

In [ ]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break
    

print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break
    
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break

Train
Text matrix size: torch.Size([133, 128])
Target vector size: torch.Size([128])

Valid:
Text matrix size: torch.Size([58, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([42, 128])
Target vector size: torch.Size([128])


### Model

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_length):
        #[sentence len, batch size] => [sentence len, batch size, embedding size]
        embedded = self.embedding(text)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, text_length.cpu())
        
        #[sentence len, batch size, embedding size] => 
        #  output: [sentence len, batch size, hidden size]
        #  hidden: [1, batch size, hidden size]
        packed_output, (hidden, cell) = self.rnn(packed)
        return self.fc(hidden.squeeze(0)).view(-1)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
torch.manual_seed(RANDOM_SEED)
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

### Training

In [ ]:
def compute_binary_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(data_loader):
            text, text_lengths = batch_data.text
            logits = model(text, text_lengths)
            predicted_labels = (torch.sigmoid(logits) > 0.5).long()
            num_examples += batch_data.label.size(0)
            correct_pred += (predicted_labels == batch_data.label.long()).sum()
        return correct_pred.float()/num_examples * 100


for epoch in range(NUM_EPOCHS):
    model.train()
    
    for batch_idx, batch_data in enumerate(train_loader):
        
      text, text_lengths = batch_data.text
        
      # FORWARD AND BACK PROP
      logits = model(text, text_lengths)
      cost = F.binary_cross_entropy_with_logits(logits, batch_data.label)
      optimizer.zero_grad()
        
      cost.backward()
        
      # UPDATE MODEL PARAMETERS
      optimizer.step()
        
      # Logging
      if not batch_idx % 50:
        print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch+1, NUM_EPOCHS, batch_idx, 
                     len(train_loader), cost))

    model.eval()
    with torch.set_grad_enabled(False):
        print('Epoch: %03d/%03d | Train: %.3f%% ' %(
              epoch+1, NUM_EPOCHS, 
              compute_binary_accuracy(model, train_loader)))

Epoch: 001/010 | Batch 0000/0157 | Cost: 0.6964
Epoch: 001/010 | Batch 0050/0157 | Cost: 0.6892
Epoch: 001/010 | Batch 0100/0157 | Cost: 0.6807
Epoch: 001/010 | Batch 0150/0157 | Cost: 0.6720
Epoch: 001/010 | Train: 59.390% 
Epoch: 002/010 | Batch 0000/0157 | Cost: 0.6640
Epoch: 002/010 | Batch 0050/0157 | Cost: 0.6537
Epoch: 002/010 | Batch 0100/0157 | Cost: 0.6171
Epoch: 002/010 | Batch 0150/0157 | Cost: 0.6121
Epoch: 002/010 | Train: 70.565% 
Epoch: 003/010 | Batch 0000/0157 | Cost: 0.5752
Epoch: 003/010 | Batch 0050/0157 | Cost: 0.5931
Epoch: 003/010 | Batch 0100/0157 | Cost: 0.5470
Epoch: 003/010 | Batch 0150/0157 | Cost: 0.6280
Epoch: 003/010 | Train: 74.690% 
Epoch: 004/010 | Batch 0000/0157 | Cost: 0.4507
Epoch: 004/010 | Batch 0050/0157 | Cost: 0.5037
Epoch: 004/010 | Batch 0100/0157 | Cost: 0.4900
Epoch: 004/010 | Batch 0150/0157 | Cost: 0.4947
Epoch: 004/010 | Train: 77.825% 
Epoch: 005/010 | Batch 0000/0157 | Cost: 0.4872
Epoch: 005/010 | Batch 0050/0157 | Cost: 0.4827
Epoc

### Evaluation

In [ ]:
import spacy
nlp = spacy.load('en')


def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [ ]:
print('Probability positive:')
predict_sentiment(model, "I really love this movie. This movie is so great!")

Probability positive:


0.7884787917137146